In [1]:
from pyspark.sql import SparkSession
# spark = SparkSession.builder.appName('LinearSVC_Pyspark').getOrCreate()
spark = SparkSession \
    .builder \
    .appName("LinearSVC_Pyspark") \
    .config("spark.executor.memory", '15G') \
    .config("spark.driver.memory", '50G') \
    .getOrCreate()




# conf = SparkConf().setAppName("LinearSVC_Pyspark")
# conf = (conf.setMaster('local[*]')
#         .set('spark.executor.memory', '4G')
#         .set('spark.driver.memory', '45G')
#         .set('spark.driver.maxResultSize', '10G'))
# spark = SparkContext(conf=conf)


spark

In [2]:
spark._conf.get('spark.driver.memory')

'50G'

In [3]:
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.ml.feature import StringIndexer
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.util import MLUtils
from pyspark.mllib.regression import LabeledPoint
from pyspark.sql.functions import lit
from math import sqrt
import numpy as np

# Calculations

In [4]:
from math import sqrt
def calculation(array):
    num_items = len(array)
    print (num_items, sum(array))
    if (num_items !=0):
        mean = sum(array) / num_items
    else :
        mean = 0
    differences = [x - mean for x in array]
    sq_differences = [d ** 2 for d in differences]
    ssd = sum(sq_differences)
    if (num_items !=1):
        variance = ssd / (num_items - 1)
    else :
        variance = ssd
    sd = sqrt(variance)
    return [mean, sd, max(array), min(array)]

calcUdf = F.udf(calculation, T.ArrayType(T.FloatType()))

In [5]:
def median(values_list):
    med = np.median(values_list)
    return float(med)
udf_median = F.udf(median, T.FloatType())

# Read Data

In [6]:
df_watch_acc = spark.read.format("csv").option("header", "true").load("Watch_accelerometer.csv")
df_watch_acc = df_watch_acc.withColumn("Sensor", lit("accelerometer"))
df_watch_acc = df_watch_acc.withColumn("Device", lit("Watch"))
df_watch_acc=df_watch_acc.select(F.col('User'), F.col('Model'),F.col('Sensor'),F.col('Device'), F.col('gt'), F.col('x').cast('float'), F.col('y').cast('float'), F.col('z').cast('float'))

#-----------------------------------------------------------------------------------------------
df_Phone_acc = spark.read.format("csv").option("header", "true").load("Phones_accelerometer.csv")
df_Phone_acc = df_Phone_acc.withColumn("Sensor", lit("accelerometer"))
df_Phone_acc= df_Phone_acc.withColumn("Device", lit("Phone"))
df_Phone_acc=df_Phone_acc.select(F.col('User'), F.col('Model'),F.col('Sensor'),F.col('Device'), F.col('gt'), F.col('x').cast('float'), F.col('y').cast('float'), F.col('z').cast('float'))

#-----------------------------------------------------------------------------------------------
df_watch_gyr = spark.read.format("csv").option("header", "true").load("Watch_gyroscope.csv")
df_watch_gyr = df_watch_gyr.withColumn("Sensor", lit("gyroscope"))
df_watch_gyr = df_watch_gyr.withColumn("Device", lit("Watch"))
df_watch_gyr=df_watch_gyr.select(F.col('User'), F.col('Model'),F.col('Sensor'),F.col('Device'), F.col('gt'), F.col('x').cast('float'), F.col('y').cast('float'), F.col('z').cast('float'))

#-----------------------------------------------------------------------------------------------
df_Phone_gyr = spark.read.format("csv").option("header", "true").load("Phones_gyroscope.csv")
df_Phone_gyr = df_Phone_gyr.withColumn("Sensor", lit("gyroscope"))
df_Phone_gyr = df_Phone_gyr.withColumn("Device", lit("Phone"))
df_Phone_gyr=df_Phone_gyr.select(F.col('User'), F.col('Model'),F.col('Sensor'),F.col('Device'), F.col('gt'), F.col('x').cast('float'), F.col('y').cast('float'), F.col('z').cast('float'))

#-----------------------------------------------------------------------------------------------
df = df_watch_acc
df = df.union(df_Phone_acc)
df = df.union(df_watch_gyr)
df = df.union(df_Phone_gyr)
df.count()


33741500

In [7]:
df.show()

+----+-----+-------------+------+-----+-----------+----------+-----------+
|User|Model|       Sensor|Device|   gt|          x|         y|          z|
+----+-----+-------------+------+-----+-----------+----------+-----------+
|   a| gear|accelerometer| Watch|stand| -0.5650316| -9.572019|-0.61411273|
|   a| gear|accelerometer| Watch|stand|-0.83258367| -9.713276|-0.60693014|
|   a| gear|accelerometer| Watch|stand| -1.0181342| -9.935339|-0.54408234|
|   a| gear|accelerometer| Watch|stand| -1.2228385|-10.142437| -0.5662287|
|   a| gear|accelerometer| Watch|stand| -1.5771804|-10.480618|-0.40282443|
|   a| gear|accelerometer| Watch|stand| -2.1643584|-10.920552|-0.18375498|
|   a| gear|accelerometer| Watch|stand|     -2.973|-11.063007| 0.21188685|
|   a| gear|accelerometer| Watch|stand| -3.8881836| -11.08276|  0.6847417|
|   a| gear|accelerometer| Watch|stand| -4.8919525|-10.890625|    1.01574|
|   a| gear|accelerometer| Watch|stand| -12.600683| -7.674015| -1.1791444|
|   a| gear|accelerometer

## Group BY  (User, Model, gt,Sensor,Device)

In [8]:
df2=df.groupBy('User', 'Model','gt','Sensor','Device')\
    .agg(calcUdf(F.collect_list(F.col('x'))).alias('x'), calcUdf(F.collect_list(F.col('y'))).alias('y'), calcUdf(F.collect_list(F.col('z'))).alias('z') , udf_median(F.collect_list(F.col('x'))).alias('medianx'),udf_median(F.collect_list(F.col('y'))).alias('mediany'),udf_median(F.collect_list(F.col('z'))).alias('medianz'))\
    .select(F.col('User'), F.col('Model'),F.col('Device'),F.col('Sensor'), F.col('gt'), F.col('x')[0].alias('meanx'), F.col('y')[0].alias('meany'), F.col('z')[0].alias('meanz'), F.col('x')[1].alias('deviationx'), F.col('y')[1].alias('deviationy'), F.col('z')[1].alias('deviationz'), F.col('x')[2].alias('maxx'), F.col('y')[2].alias('maxy'), F.col('z')[2].alias('maxz'), F.col('x')[3].alias('minx'), F.col('y')[3].alias('miny'), F.col('z')[3].alias('minz'),F.col('medianx'),F.col('mediany'),F.col('medianz'))

## StringIndexer

In [9]:
indexer = StringIndexer(inputCol="gt", outputCol="label")
df3 = indexer.fit(df2).transform(df2)
indexer2 = StringIndexer(inputCol="Model", outputCol="ModelIndex")
df4 = indexer2.fit(df3).transform(df3)
indexer3 = StringIndexer(inputCol="Sensor", outputCol="SensorIndex")
df5 = indexer3.fit(df4).transform(df4)
indexer4 = StringIndexer(inputCol="Device", outputCol="DeviceIndex")
df6 = indexer4.fit(df5).transform(df5)


In [10]:
inputData = df6.select('label', "meanx", "meany", 'meanz', 'deviationx', 'deviationy', 'deviationz', 'maxx', 'maxy', 'maxz','minx','miny','minz','medianx','mediany','medianz','ModelIndex','SensorIndex','DeviceIndex')

In [11]:
inputData.count()

678

In [12]:
inputData.printSchema()

root
 |-- label: double (nullable = false)
 |-- meanx: float (nullable = true)
 |-- meany: float (nullable = true)
 |-- meanz: float (nullable = true)
 |-- deviationx: float (nullable = true)
 |-- deviationy: float (nullable = true)
 |-- deviationz: float (nullable = true)
 |-- maxx: float (nullable = true)
 |-- maxy: float (nullable = true)
 |-- maxz: float (nullable = true)
 |-- minx: float (nullable = true)
 |-- miny: float (nullable = true)
 |-- minz: float (nullable = true)
 |-- medianx: float (nullable = true)
 |-- mediany: float (nullable = true)
 |-- medianz: float (nullable = true)
 |-- ModelIndex: double (nullable = false)
 |-- SensorIndex: double (nullable = false)
 |-- DeviceIndex: double (nullable = false)



## Round

In [13]:
inputData = inputData.withColumn('meanx',F.round('meanx',4))
inputData = inputData.withColumn('meany',F.round('meany',4))
inputData = inputData.withColumn('meanz',F.round('meanz',4))
inputData = inputData.withColumn('deviationx',F.round('deviationx',4))
inputData = inputData.withColumn('deviationy',F.round('deviationy',4))
inputData = inputData.withColumn('deviationz',F.round('deviationz',4))
inputData = inputData.withColumn('maxx',F.round('maxx',4))
inputData = inputData.withColumn('maxy',F.round('maxy',4))
inputData = inputData.withColumn('maxz',F.round('maxz',4))
inputData = inputData.withColumn('minx',F.round('minx',4))
inputData = inputData.withColumn('miny',F.round('miny',4))
inputData = inputData.withColumn('minz',F.round('minz',4))
inputData = inputData.withColumn('medianx',F.round('medianx',4))
inputData = inputData.withColumn('mediany',F.round('mediany',4))
inputData = inputData.withColumn('medianz',F.round('medianz',4))

inputData.show(5)

+-----+-------+-------+------+----------+----------+----------+-------+------+-------+--------+--------+-------+-------+-------+-------+----------+-----------+-----------+
|label|  meanx|  meany| meanz|deviationx|deviationy|deviationz|   maxx|  maxy|   maxz|    minx|    miny|   minz|medianx|mediany|medianz|ModelIndex|SensorIndex|DeviceIndex|
+-----+-------+-------+------+----------+----------+----------+-------+------+-------+--------+--------+-------+-------+-------+-------+----------+-----------+-----------+
|  0.0| -7.392|-0.8664|6.0925|     1.417|    1.4106|    1.0939| 0.1203|7.4945|13.5805|-19.6133|-10.5842|-2.9239|-7.3897| -0.978| 6.1537|       4.0|        0.0|        1.0|
|  3.0|-9.5261| -3.127|0.1902|    5.6505|    2.1909|    2.5229|16.2967|7.0204|11.9997|-19.6133|-10.8625|-8.1588|-10.413|-3.0658|  0.097|       4.0|        0.0|        1.0|
|  0.0|-4.4658| 0.6265|8.4112|    1.1281|    1.4526|    0.6795|-0.4693|6.3973|13.1873| -8.3414| -3.3902| 4.7118|-4.6831|  0.182|  8.418|    

In [14]:
inputData = inputData.na.drop()

## Libsvm Format

In [15]:
c = inputData.rdd
d = c.map(lambda line: LabeledPoint(line[0],line[1:]))

In [17]:
MLUtils.saveAsLibSVMFile(d,"libsvm")

In [18]:
spark.sparkContext.textFile("libsvm" + "/part-*").coalesce(1).saveAsTextFile("libsvm/data")

In [19]:
loadedDataDF=spark.read.format("libsvm").load("libsvm/data/part-00000")

# Model

In [55]:
from pyspark.ml.classification import LinearSVC, OneVsRest
from pyspark.ml.evaluation import MulticlassClassificationEvaluator


(train, test) = loadedDataDF.randomSplit([0.67, 0.33])
lsvc = LinearSVC(maxIter=15, regParam=0.01)
ovr = OneVsRest(classifier=lsvc)
ovrModel = ovr.fit(train)
predictions = ovrModel.transform(test)

## Evaluator

In [56]:
evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print("Test Error = %g" % (1.0 - accuracy))
print("Accuracy = %g" % accuracy)

Test Error = 0.582734
Accuracy = 0.417266


In [57]:
predictions.select('label','prediction').show()

+-----+----------+
|label|prediction|
+-----+----------+
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       1.0|
|  0.0|       4.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       4.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  0.0|       6.0|
|  0.0|       6.0|
|  0.0|       0.0|
|  0.0|       0.0|
|  1.0|       3.0|
|  1.0|       3.0|
+-----+----------+
only showing top 20 rows

